# 9. 모델링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터 준비

In [2]:
data = pd.read_csv("C:/Users/Yoo/Documents/Capstone/실거래(13~18).csv", thousands = ',')
district = pd.read_excel("C:/Users/Yoo/Documents/Capstone/클러스터링_hierarchycal.xlsx")

C:\Users\Yoo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
uniq_dis = district.area.str[:-2].unique()

In [4]:
AA = data.행정동
BB = []

for i in range(len(AA)):
    if AA[i] in uniq_dis:
        BB.append(i)

In [5]:
data = data.loc[BB]

In [6]:
data = data.dropna().reset_index(drop=True)

In [7]:
new_index = np.random.permutation(len(data))
data = data.iloc[new_index].reset_index(drop=True)

In [8]:
data.rename(columns={'거래금액':'price', '건축년도':'con_year', '년':'year', '월':'month', '일':'day', '전용면적':'area', '층':'layer'}, inplace=True)

In [9]:
for i in range(len(data)):
    if data.at[i,'month']<4:
        data.at[i,'quarter'] = str(data.at[i,'year']) + '1/4'
    elif data.at[i,'month']>3 and data.at[i,'month']<7:
        data.at[i,'quarter'] = str(data.at[i,'year']) + '2/4'
    elif data.at[i,'month']>6 and data.at[i,'month']<10:
        data.at[i,'quarter'] = str(data.at[i,'year']) + '3/4'
    else:
        data.at[i,'quarter'] = str(data.at[i,'year']) + '4/4'
        
quantile = data['quarter'].unique()
number_list = np.arange(24)
match = (quantile, number_list)

def match_quantile(quantile):
    a = np.where(match[0] == quantile)
    return number_list[a[0][0]]

data['time'] = data['quarter'].apply(lambda x: match_quantile(x))
data.drop(['quarter'], axis='columns', inplace=True)

In [10]:
data_1 =  data[['con_year','time','area','layer','land_price','D_hsch','D_msch','D_esch','D_sub','D_trans','D_hos','D_market',
          'D_library','D_factory','E_GDP','E_grow','E_CPI','E_rent','E_kospi','E_rate','A_benefit','A_pop_all','A_pop_65','A_pop_for',
          'A_par_all','A_par_apt','brand','Max_layer','A_top','A_bottom','price','행정동','lat','lng']]

In [11]:
corr_matrix = data_1.corr().abs()

In [12]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [13]:
threshold = 0.7
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(to_drop)))
to_drop

There are 3 columns to remove.


['E_rent', 'E_kospi', 'A_pop_65']

In [24]:
col_list = {'con_year','time','area','layer','land_price','D_hsch','D_msch','D_esch','D_sub','D_trans','D_hos','D_market',
          'D_library','D_factory','E_GDP','E_grow','E_CPI','E_rent','E_kospi','E_rate','A_benefit','A_pop_all','A_pop_65','A_pop_for',
          'A_par_all','A_par_apt','brand','Max_layer','A_top','A_bottom','price','행정동','lat','lng', '도로명', '도로명건물본번호코드',
           '도로명건물부번호코드','도로명시군구코드','아파트'}

In [25]:
data = data[col_list.difference(to_drop)]

In [28]:
for i in uniq_dis:
    A, B = np.percentile(data.loc[data['행정동'] == i, 'area'], [30, 70])
    
    temp = data[data['행정동']==i]
    
    X = temp.index
    X1 = temp[temp['area'] < A].index
    X2 = temp[temp['area'] > B].index
    X = X.difference(X1).difference(X2)
    
    data.loc[X, 'cluster'] = '{0}_중'.format(i)
    data.loc[X1, 'cluster'] = '{0}_소'.format(i)
    data.loc[X2, 'cluster'] = '{0}_대'.format(i)

In [29]:
area_index = list(district['area'])
class_index = list(district['class'])

for i, j in enumerate(data['cluster']):
    num = area_index.index(j)
    
    data.at[i,'class'] = class_index[num]

In [30]:
def outliers(value,q1,q3):
    low_limit = q1 - 3*(q3-q1)
    high_limit = q3 + 3*(q3-q1)
    
    if value >= low_limit and value <= high_limit:
        return 0
    else: 
        return 1

In [31]:
old_index = list(data.cluster.unique())
new_index = []
new_index_q = []

for i in old_index:
    new_index.append(data[data['cluster'] == i].index)

for i in range(len(new_index)):
    new_index_q.append(np.percentile(data.loc[new_index[i],'price'], [25,75]))
    data.loc[new_index[i],'outlier'] = i

In [32]:
for i in range(len(data)):
    ob_num = int(data.at[i, 'outlier'])
    data.at[i, 'isout'] = outliers(data.at[i,'price'], new_index_q[ob_num][0], new_index_q[ob_num][1])

In [33]:
data.isout.sum()

779.0

In [34]:
data = data[data['isout'] == 0]
data.drop(['outlier', 'isout'], axis=1, inplace=True)

C:\Users\Yoo\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
data_class0 = data[data['class'] == 0]
data_class1 = data[data['class'] == 1]
data_class2 = data[data['class'] == 2]

In [36]:
data_class0.to_excel('C:/Users/Yoo/Documents/Capstone/data_class0.xlsx')
data_class1.to_excel('C:/Users/Yoo/Documents/Capstone/data_class1.xlsx')
data_class2.to_excel('C:/Users/Yoo/Documents/Capstone/data_class2.xlsx')